In [21]:
import os

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"
os.environ["AWS_ACCESS_KEY_ID"] = "minio_user"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio_password"

In [22]:
import pandas as pd

In [23]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor

In [24]:
import mlflow

# MLFLOW_TRACKING_URI = "http://ec2-3-123-1-12.eu-central-1.compute.amazonaws.com:5000"

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("module_4_nyc_taxi_duration_prediction_with_sklearn_pipeline_experiment")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1750103452800, experiment_id='1', last_update_time=1750103452800, lifecycle_stage='active', name='module_4_nyc_taxi_duration_prediction_with_sklearn_pipeline_experiment', tags={}>

In [25]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df

In [26]:
def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [27]:
train_file = '../data/green_tripdata_2025-01.parquet'
val_file = '../data/green_tripdata_2025-02.parquet'

df_train = read_dataframe(train_file)
df_val = read_dataframe(val_file)

In [28]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [29]:
dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [30]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_split=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(), 
        RandomForestRegressor(**params, n_jobs=-1)
    )
    pipeline.fit(dict_train, y_train)

    y_pred = pipeline.predict(dict_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

2025/06/16 22:38:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run rebellious-pig-837 at: http://localhost:5000/#/experiments/1/runs/0d228fa7f2a04073a1e097c5a35b55c5
🧪 View experiment at: http://localhost:5000/#/experiments/1
